# Duelling-DQN: CartPole-v0 | EPOCH Lab

A pole is attached by an un-actuated joint to a cart, which moves along a frictionless track. The system is controlled by applying a force of +1 or -1 to the cart. The pendulum starts upright, and the goal is to prevent it from falling over. A reward of +1 is provided for every timestep that the pole remains upright. The episode ends when the pole is more than 15 degrees from vertical, or the cart moves more than 2.4 units from the center.

- Python: 3.6.12
- Keras-GPU: 2.3.1
- KerasRL2: 1.0.4

In [1]:
import gym

import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.optimizers import Adam

from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

### Build OpenAI Gym Environment

Get the environment and extract the number of states and actions.

In [2]:
ENV_NAME = 'MountainCar-v0'

In [3]:
env = gym.make(ENV_NAME)
np.random.seed(123)
env.seed(123)

/home/james/anaconda3/envs/gym/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


[123]

In [4]:
states = env.observation_space.shape
actions = env.action_space.n

print('States:', states[0])
print('Actions:', actions)

States: 2
Actions: 3


### Create Deep Learning Model

Build a very simple model regardless of the dueling architecture if you enable dueling network in DQN , DQN will build a dueling network base on your model automatically. Also, you can build a dueling network by yourself and turn off the dueling network in DQN.

In [5]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape = (1, ) + states))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(actions, activation = 'linear'))
    
    return model

In [6]:
model = build_model(states, actions)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 2)                 0         
_________________________________________________________________
dense (Dense)                (None, 128)               384       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 99        
Total params: 10,819
Trainable params: 10,819
Non-trainable params: 0
_________________________________________________________________


Configure and compile our agent. You can use every built-in tensorflow.keras optimizer and even the metrics.

In [7]:
memory = SequentialMemory(limit=50000, window_length=1)
policy = BoltzmannQPolicy()

dqn = DQNAgent(model=model, nb_actions=actions, memory=memory, nb_steps_warmup=10, target_model_update=1e-2, policy=policy)
#duel_dqn = DQNAgent(model=model, nb_actions=actions, memory=memory, nb_steps_warmup=10, enable_dueling_network=True, dueling_type='avg', target_model_update=1e-2, policy=policy)

dqn.compile(Adam(lr=1e-3), metrics=['mae'])

### Training Loop

In [8]:
dqn.fit(env, nb_steps=150000, visualize=False, verbose=2)

Training for 150000 steps ...


/home/james/anaconda3/envs/gym/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
/home/james/anaconda3/envs/gym/lib/python3.7/site-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


    200/150000: episode: 1, duration: 4.311s, episode steps: 200, steps per second:  46, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 1.130 [0.000, 2.000],  loss: 0.061807, mae: 0.814149, mean_q: -1.055888
    400/150000: episode: 2, duration: 3.830s, episode steps: 200, steps per second:  52, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 1.060 [0.000, 2.000],  loss: 0.006198, mae: 1.768291, mean_q: -2.607878
    600/150000: episode: 3, duration: 3.803s, episode steps: 200, steps per second:  53, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 1.080 [0.000, 2.000],  loss: 0.026289, mae: 2.863523, mean_q: -4.214609
    800/150000: episode: 4, duration: 3.896s, episode steps: 200, steps per second:  51, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 0.995 [0.000, 2.000],  loss: 0.050789, mae: 3.993084, mean_q: -5.883262
   1000/150000: episode: 5, duration: 3.807s, ep

   7200/150000: episode: 36, duration: 3.760s, episode steps: 200, steps per second:  53, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 1.150 [0.000, 2.000],  loss: 3.302649, mae: 26.229649, mean_q: -38.808105
   7400/150000: episode: 37, duration: 3.671s, episode steps: 200, steps per second:  54, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 1.110 [0.000, 2.000],  loss: 2.956753, mae: 26.641050, mean_q: -39.466808
   7600/150000: episode: 38, duration: 3.522s, episode steps: 200, steps per second:  57, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 0.975 [0.000, 2.000],  loss: 4.120373, mae: 27.049522, mean_q: -39.955265
   7800/150000: episode: 39, duration: 3.638s, episode steps: 200, steps per second:  55, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 1.165 [0.000, 2.000],  loss: 3.097409, mae: 27.403852, mean_q: -40.536224
   8000/150000: episode: 40, duratio

  14000/150000: episode: 70, duration: 3.767s, episode steps: 200, steps per second:  53, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 0.985 [0.000, 2.000],  loss: 7.310924, mae: 35.670403, mean_q: -52.835567
  14200/150000: episode: 71, duration: 3.767s, episode steps: 200, steps per second:  53, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 1.085 [0.000, 2.000],  loss: 8.669079, mae: 35.754822, mean_q: -52.918766
  14400/150000: episode: 72, duration: 3.841s, episode steps: 200, steps per second:  52, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 0.935 [0.000, 2.000],  loss: 4.750194, mae: 35.933838, mean_q: -53.359482
  14600/150000: episode: 73, duration: 3.739s, episode steps: 200, steps per second:  53, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 1.005 [0.000, 2.000],  loss: 7.724871, mae: 36.065506, mean_q: -53.361401
  14800/150000: episode: 74, duratio

...

 122759/150000: episode: 614, duration: 3.747s, episode steps: 200, steps per second:  53, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 1.170 [0.000, 2.000],  loss: 2.648351, mae: 28.687651, mean_q: -42.217628
 122959/150000: episode: 615, duration: 3.587s, episode steps: 200, steps per second:  56, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 0.970 [0.000, 2.000],  loss: 2.494811, mae: 28.654043, mean_q: -42.207588
 123159/150000: episode: 616, duration: 3.738s, episode steps: 200, steps per second:  54, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 1.045 [0.000, 2.000],  loss: 3.203196, mae: 28.650459, mean_q: -42.167778
 123359/150000: episode: 617, duration: 3.588s, episode steps: 200, steps per second:  56, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 1.005 [0.000, 2.000],  loss: 2.384006, mae: 28.508276, mean_q: -41.989922
 123559/150000: episode: 618, du

 129559/150000: episode: 648, duration: 3.670s, episode steps: 200, steps per second:  54, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 1.025 [0.000, 2.000],  loss: 3.206414, mae: 27.745073, mean_q: -40.791969
 129759/150000: episode: 649, duration: 3.605s, episode steps: 200, steps per second:  55, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 1.030 [0.000, 2.000],  loss: 2.140985, mae: 27.747509, mean_q: -40.839123
 129959/150000: episode: 650, duration: 3.664s, episode steps: 200, steps per second:  55, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 0.990 [0.000, 2.000],  loss: 2.302392, mae: 27.739817, mean_q: -40.787045
 130159/150000: episode: 651, duration: 3.709s, episode steps: 200, steps per second:  54, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 1.020 [0.000, 2.000],  loss: 2.929875, mae: 27.671612, mean_q: -40.732922
 130359/150000: episode: 652, du

 136354/150000: episode: 682, duration: 3.671s, episode steps: 200, steps per second:  54, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 1.020 [0.000, 2.000],  loss: 2.034496, mae: 26.897520, mean_q: -39.488964
 136554/150000: episode: 683, duration: 3.658s, episode steps: 200, steps per second:  55, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 1.020 [0.000, 2.000],  loss: 2.083821, mae: 27.029882, mean_q: -39.710655
 136754/150000: episode: 684, duration: 3.627s, episode steps: 200, steps per second:  55, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 1.015 [0.000, 2.000],  loss: 2.043050, mae: 26.975658, mean_q: -39.614975
 136954/150000: episode: 685, duration: 3.693s, episode steps: 200, steps per second:  54, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 0.835 [0.000, 2.000],  loss: 1.942770, mae: 26.993130, mean_q: -39.618088
 137154/150000: episode: 686, du

 143131/150000: episode: 716, duration: 3.704s, episode steps: 200, steps per second:  54, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 1.085 [0.000, 2.000],  loss: 1.852261, mae: 27.201763, mean_q: -39.980240
 143331/150000: episode: 717, duration: 3.704s, episode steps: 200, steps per second:  54, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 1.015 [0.000, 2.000],  loss: 1.855474, mae: 27.209457, mean_q: -40.017262
 143531/150000: episode: 718, duration: 3.678s, episode steps: 200, steps per second:  54, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 1.015 [0.000, 2.000],  loss: 1.470989, mae: 27.217056, mean_q: -40.029129
 143731/150000: episode: 719, duration: 3.581s, episode steps: 200, steps per second:  56, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 0.950 [0.000, 2.000],  loss: 1.362884, mae: 27.279579, mean_q: -40.099075
 143931/150000: episode: 720, du

 149931/150000: episode: 750, duration: 3.651s, episode steps: 200, steps per second:  55, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 1.015 [0.000, 2.000],  loss: 1.326276, mae: 26.500669, mean_q: -38.882103
done, took 2728.478 seconds


In [9]:
# After training is done, we save the final weights.
dqn.save_weights('results/dqn_mountaincar_{}_weights.h5f'.format(ENV_NAME), overwrite=True)

### Inference

In [11]:
# Finally, evaluate our algorithm for 5 episodes.
dqn.test(env, nb_episodes=5, visualize=True)

Testing for 5 episodes ...
Episode 1: reward: -116.000, steps: 116
Episode 2: reward: -172.000, steps: 172
Episode 3: reward: -162.000, steps: 162
Episode 4: reward: -117.000, steps: 117
Episode 5: reward: -156.000, steps: 156
